# 第4章: 混合整数線形計画法（MILP）(Julia版)

この章では、整数変数と二値変数を含む混合整数線形計画法を学びます。

In [1]:
using JuMP
using HiGHS

## 4.1 整数変数と二値変数の基本

JuMPでは `@variable` マクロで変数の種類を指定できます。

### 変数の種類

| 種類 | 説明 | JuMPでの定義 |
|------|------|-------------|
| 連続変数 | デフォルト、任意の実数値 | `@variable(model, x)` |
| 整数変数 | 整数値のみ | `@variable(model, x, Int)` |
| 二値変数 | 0または1のみ | `@variable(model, x, Bin)` |

### 二値変数の用途

- Yes/No の意思決定
- 論理条件の表現
- 選択問題のモデル化

## 例題1: 整数変数を含む生産計画

### 問題

工場で2種類の製品を生産します。製品は「個」単位でしか生産できません（整数制約）。

|  | 利益 | 機械時間 | 人員 |
|--|------|----------|------|
| 製品A | \$50 | 2時間 | 3人 |
| 製品B | \$40 | 3時間 | 2人 |

**制約：**
- 機械時間: 1日12時間まで
- 人員: 1日10人まで

**目標:** 利益を最大化する生産量（整数）を求める

In [2]:
model = Model(HiGHS.Optimizer)
set_silent(model)

# 整数変数として定義
@variable(model, x_a >= 0, Int)  # 製品A（整数）
@variable(model, x_b >= 0, Int)  # 製品B（整数）

# 制約条件
@constraint(model, 2x_a + 3x_b <= 12)  # 機械時間
@constraint(model, 3x_a + 2x_b <= 10)  # 人員

# 目的関数
@objective(model, Max, 50x_a + 40x_b)

optimize!(model)

println("【整数変数の結果】")
println("ステータス: ", termination_status(model))
println("最大利益: \$", round(Int, objective_value(model)))
println("\n生産量:")
println("  製品A: ", round(Int, value(x_a)), " 個")
println("  製品B: ", round(Int, value(x_b)), " 個")

【整数変数の結果】


ステータス: OPTIMAL
最大利益: $

180

生産量:
  製品A: 2 個
  製品B: 2 個


In [3]:
# 連続変数の場合と比較
println("【参考】連続変数の場合:")
model_cont = Model(HiGHS.Optimizer)
set_silent(model_cont)
@variable(model_cont, x_a_cont >= 0)
@variable(model_cont, x_b_cont >= 0)
@constraint(model_cont, 2x_a_cont + 3x_b_cont <= 12)
@constraint(model_cont, 3x_a_cont + 2x_b_cont <= 10)
@objective(model_cont, Max, 50x_a_cont + 40x_b_cont)
optimize!(model_cont)
println("  製品A: ", round(value(x_a_cont), digits=2), " 個")
println("  製品B: ", round(value(x_b_cont), digits=2), " 個")
println("  最大利益: \$", round(objective_value(model_cont), digits=2))
println("  → 整数制約により、解が異なる場合があります")

【参考】連続変数の場合:
  製品A: 1.2

 個
  製品B: 3.2 個
  最大利益: $188.0
  → 整数制約により、解が異なる場合があります


## 例題2: プロジェクト選択問題

### 問題

5つのプロジェクト候補があり、予算\$100,000でどれを実行するか決定します。

| プロジェクト | コスト | 期待利益 |
|--------------|--------|----------|
| A | \$30,000 | \$50,000 |
| B | \$40,000 | \$60,000 |
| C | \$25,000 | \$35,000 |
| D | \$35,000 | \$55,000 |
| E | \$20,000 | \$25,000 |

**目標:** 予算内で期待利益を最大化

In [4]:
model = Model(HiGHS.Optimizer)
set_silent(model)

# プロジェクトデータ
projects = [:A, :B, :C, :D, :E]
cost = Dict(:A => 30000, :B => 40000, :C => 25000, :D => 35000, :E => 20000)
profit = Dict(:A => 50000, :B => 60000, :C => 35000, :D => 55000, :E => 25000)
budget = 100000

# 二値変数（実行する=1, しない=0）
@variable(model, x[projects], Bin)

# 予算制約
@constraint(model, sum(cost[p] * x[p] for p in projects) <= budget)

# 目的関数
@objective(model, Max, sum(profit[p] * x[p] for p in projects))

optimize!(model)

println("【結果】")
println("ステータス: ", termination_status(model))
println("期待総利益: \$", round(Int, objective_value(model)))
println("\n選択されたプロジェクト:")
total_cost = 0
for p in projects
    if value(x[p]) > 0.5
        println("  $p: コスト \$", cost[p], ", 利益 \$", profit[p])
        total_cost += cost[p]
    end
end
println("\n使用予算: \$", total_cost, " / \$", budget)

【結果】


ステータス: OPTIMAL
期待総利益: $150000

選択されたプロジェクト:
  B: コスト $40000, 利益 $60000
  C: コスト $25000, 利益 $35000
  D: コスト $35000, 利益 $55000

使用予算: $100000 / $100000


## 例題3: 論理制約（排他的選択）

### 問題

リソース配分問題に論理制約を追加します。

機械の制約により、**製品1と製品3は同時に生産できません**。

### 技法: Big-M法

二値変数 y を使って、排他的制約を表現します：
- $x_1 \leq M \cdot y_1$（$y_1=0$ なら $x_1=0$）
- $x_3 \leq M \cdot y_3$（$y_3=0$ なら $x_3=0$）
- $y_1 + y_3 \leq 1$（$y_1$か$y_3$の一方のみ1）

In [5]:
model = Model(HiGHS.Optimizer)
set_silent(model)

# 製品の生産量（連続変数）
@variable(model, x[1:4] >= 0)

# 製品1と3の選択を表す二値変数
@variable(model, y[i in [1, 3]], Bin)

# 利益係数
profit = [20, 12, 40, 25]

# 通常の制約
@constraint(model, x[1] + x[2] + x[3] + x[4] <= 50)  # 人員
@constraint(model, 3x[1] + 2x[2] + x[3] <= 100)       # 原材料A
@constraint(model, x[2] + 2x[3] + 3x[4] <= 90)        # 原材料B

# Big-M法による論理制約
M = 100  # 十分大きな数
@constraint(model, x[1] <= M * y[1])
@constraint(model, x[3] <= M * y[3])
@constraint(model, y[1] + y[3] <= 1)  # 排他的制約

# 目的関数
@objective(model, Max, sum(profit[i] * x[i] for i in 1:4))

optimize!(model)

println("【結果】")
println("ステータス: ", termination_status(model))
println("最大利益: \$", round(objective_value(model), digits=2))
println("\n生産計画:")
for i in 1:4
    println("  製品$i: ", round(value(x[i]), digits=2), " 個")
end
println("\n選択:")
println("  製品1を生産: ", value(y[1]) > 0.5 ? "はい" : "いいえ")
println("  製品3を生産: ", value(y[3]) > 0.5 ? "はい" : "いいえ")

【結果】


ステータス: OPTIMAL
最大利益: $1800.0

生産計画:
  製品1: 0.0 個
  製品2: 0.0 個
  製品3: 45.0 個
  製品4: 0.0 個

選択:
  製品1を生産: いいえ
  製品3を生産: はい


## 例題4: 固定費用を含む生産計画

### 問題

各製品ラインには、稼働させると固定費用がかかります。

|  | 変動利益/個 | 固定費用 | 最大生産量 |
|--|-------------|----------|------------|
| 製品A | \$10 | \$50 | 20個 |
| 製品B | \$15 | \$80 | 15個 |
| 製品C | \$8 | \$30 | 25個 |

**総生産時間:** 30時間まで（各製品の生産時間: 1時間/個）

**目標:** 純利益を最大化

In [6]:
model = Model(HiGHS.Optimizer)
set_silent(model)

products = [:A, :B, :C]
var_profit = Dict(:A => 10, :B => 15, :C => 8)
fixed_cost = Dict(:A => 50, :B => 80, :C => 30)
max_prod = Dict(:A => 20, :B => 15, :C => 25)

# 生産量（整数変数）
@variable(model, x[p in products] >= 0, Int)

# 製品ラインの稼働フラグ（二値変数）
@variable(model, y[products], Bin)

# 総生産時間制約
@constraint(model, sum(x[p] for p in products) <= 30)

# 生産量と稼働フラグの関係
for p in products
    @constraint(model, x[p] <= max_prod[p] * y[p])
end

# 目的関数（変動利益 - 固定費用）
@objective(model, Max, sum(var_profit[p] * x[p] - fixed_cost[p] * y[p] for p in products))

optimize!(model)

println("【結果】")
println("ステータス: ", termination_status(model))
println("純利益: \$", round(Int, objective_value(model)))
println("\n生産計画:")
for p in products
    status = value(y[p]) > 0.5 ? "稼働" : "停止"
    println("  製品$p: ", round(Int, value(x[p])), "個 ($status)")
end

# 内訳
println("\n利益の内訳:")
total_var = sum(var_profit[p] * value(x[p]) for p in products)
total_fix = sum(fixed_cost[p] * value(y[p]) for p in products)
println("  変動利益: \$", round(Int, total_var))
println("  固定費用: \$", round(Int, total_fix))
println("  純利益: \$", round(Int, total_var - total_fix))

【結果】


ステータス: OPTIMAL
純利益: $245

生産計画:
  製品A: 15個 (稼働)
  製品B: 15個 (稼働)
  製品C: 0個 (停止)

利益の内訳:
  変動利益: $375
  固定費用: $130
  純利益: $245


## 例題5: 施設配置問題

### 問題

3つの候補地から倉庫を選び、4つの店舗に製品を供給します。

**倉庫の建設コストと供給能力:**
- 候補地1: \$500,000, 能力100
- 候補地2: \$400,000, 能力80
- 候補地3: \$600,000, 能力120

**各店舗の需要:** 店舗A=40, 店舗B=50, 店舗C=30, 店舗D=60

**目標:** 総コスト（建設+輸送）を最小化

In [7]:
model = Model(HiGHS.Optimizer)
set_silent(model)

# データ
warehouses = [1, 2, 3]
stores = [:A, :B, :C, :D]

build_cost = Dict(1 => 500000, 2 => 400000, 3 => 600000)
capacity = Dict(1 => 100, 2 => 80, 3 => 120)
demand = Dict(:A => 40, :B => 50, :C => 30, :D => 60)

transport_cost = Dict(
    (1, :A) => 8, (1, :B) => 10, (1, :C) => 6, (1, :D) => 7,
    (2, :A) => 5, (2, :B) => 6, (2, :C) => 9, (2, :D) => 11,
    (3, :A) => 9, (3, :B) => 7, (3, :C) => 4, (3, :D) => 5
)

# 決定変数
@variable(model, y[warehouses], Bin)  # 倉庫を建設するか
@variable(model, x[w in warehouses, s in stores] >= 0)  # 輸送量

# 需要充足制約
for s in stores
    @constraint(model, sum(x[w, s] for w in warehouses) >= demand[s])
end

# 供給能力制約（倉庫が建設された場合のみ供給可能）
for w in warehouses
    @constraint(model, sum(x[w, s] for s in stores) <= capacity[w] * y[w])
end

# 目的関数
@objective(model, Min,
    sum(build_cost[w] * y[w] for w in warehouses) +
    sum(transport_cost[w, s] * x[w, s] for w in warehouses, s in stores))

optimize!(model)

println("【結果】")
println("ステータス: ", termination_status(model))
println("総コスト: \$", round(Int, objective_value(model)))

println("\n倉庫建設:")
for w in warehouses
    status = value(y[w]) > 0.5 ? "建設" : "建設しない"
    println("  候補地$(w): $(status)")
end

println("\n輸送計画:")
for w in warehouses
    if value(y[w]) > 0.5
        println("  候補地$(w)から:")
        for s in stores
            v = value(x[w, s])
            if v > 0
                println("    → 店舗$(s): ", round(Int, v), "個")
            end
        end
    end
end

【結果】


ステータス: OPTIMAL
総コスト: $901130

倉庫建設:
  候補地1: 建設
  候補地2: 建設
  候補地3: 建設しない

輸送計画:
  候補地1から:
    → 店舗A: 10個
    → 店舗C: 30個
    → 店舗D: 60個
  候補地2から:
    → 店舗A: 30個
    → 店舗B: 50個


## 例題6: ロットサイズ決定問題

### 問題

4期間の生産計画を立てます。

**期間ごとの需要:** 期間1=40, 期間2=60, 期間3=30, 期間4=50

**コスト:**
- 生産セットアップ費用: \$100/回
- 生産費用: \$5/個
- 在庫保管費用: \$2/個/期間

**制約:**
- 各期間の最大生産量: 80個

In [8]:
model = Model(HiGHS.Optimizer)
set_silent(model)

periods = 1:4
demand = [40, 60, 30, 50]

setup_cost = 100
prod_cost = 5
hold_cost = 2
max_prod = 80

# 決定変数
@variable(model, x[periods] >= 0, Int)  # 生産量
@variable(model, y[periods], Bin)       # セットアップ
@variable(model, inv[periods] >= 0)     # 在庫

# 在庫バランス制約
@constraint(model, x[1] - demand[1] == inv[1])  # 初期在庫0
for t in 2:4
    @constraint(model, inv[t-1] + x[t] - demand[t] == inv[t])
end

# セットアップ制約
for t in periods
    @constraint(model, x[t] <= max_prod * y[t])
end

# 目的関数
@objective(model, Min, sum(setup_cost * y[t] + prod_cost * x[t] + hold_cost * inv[t] for t in periods))

optimize!(model)

println("【結果】")
println("ステータス: ", termination_status(model))
println("総コスト: \$", round(Int, objective_value(model)))

println("\n期間別計画:")
println("期間  需要  生産  在庫  セットアップ")
println("-" ^ 45)
for t in periods
    setup = value(y[t]) > 0.5 ? "○" : "-"
    println("  $t    ", lpad(demand[t], 3), "   ",
            lpad(round(Int, value(x[t])), 3), "   ",
            lpad(round(Int, value(inv[t])), 3), "      $setup")
end

【結果】


ステータス: OPTIMAL
総コスト: $1280

期間別計画:
期間  需要  生産  在庫  セットアップ
---------------------------------------------
  1     40    50    10      ○
  2     60    80    30      ○
  3     30     0     0      -
  4     50    50     0      ○


## 第4章のまとめ

この章で学んだこと：

1. **変数の種類**
   - 連続変数: デフォルト
   - 整数変数: `Int`
   - 二値変数: `Bin`

2. **二値変数の活用**
   - Yes/No の意思決定
   - プロジェクト選択問題
   - 施設配置問題

3. **Big-M法による論理制約**
   - 排他的選択（AかBのどちらか一方）
   - 条件付き制約

4. **固定費用の扱い**
   - 二値変数で稼働/非稼働を表現
   - 変動費用と固定費用を分離

---

**次の章:** さらに多くの実践的な例題を学びます！